# JPX Competition Metric - Sharpe Ratio with constraints
We are asked to construct a portfolio that maximize Sharpe Ratio:
$$ SharpeRatio = \frac{Average(R_t)}{STD(R_t)} $$
$$ R_t  = \sum_i(r_{i,t}*w_{i,t})) $$

Return of stock at day t is calculated with the difference of close prices(C) at day t+1 from day t+2
$$ r_{i,t} = \frac{C_{i, t+2} - C_{i, t+1}} {C_{i, t+1}} $$

We pick for each day a ranking Rho from N stocks
 $$ w_{i,t} = \begin{cases} 
     2 - \frac{\rho_{i,t}} {200} & \text{if } \rho_{i,t} < 200 \\
     -2 + \frac{N_{t} - \rho_{i,t}} {200} & \text{if } N_{t} - \rho_{i,t} < 200 \\
     0 & \text{otherwise} 
 \end{cases} $$

# Baseline - Efficient Frontier without constraints
To create a simple baseline, we will consider a portfolio that does not constraint stock weights.
We will also not consider reallocating between days, i.e.
$$ w_{i,t} = w_{i} $$
We will use PyPortfolioOpt to do this.

In [ ]:
import numpy as np
np.random.seed(2022)

import pandas as pd
pd.set_option('display.max_columns', None)

from tqdm import tqdm
tqdm.pandas() 

!pip install PyPortfolioOpt

In [ ]:
#Load data
path = '../input/jpx-tokyo-stock-exchange-prediction'
stocks = pd.read_csv(f'{path}/stock_list.csv').rename(columns={'Close':'CloseTradeDate'})
stocks = stocks[stocks['Universe0']==True] #only consider the 2000 stocks
display(stocks)

prices = pd.concat([pd.read_csv(f'{path}/train_files/{file}') for file in ['stock_prices.csv']]) #, 'secondary_stock_prices.csv'
prices = prices.merge(stocks, left_on='SecuritiesCode', right_on='SecuritiesCode')
display(prices)

In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import plotting

#pivot prices to arrange by date
prices_pivot = prices[['SecuritiesCode','Date','Close']].pivot(index='Date',columns='SecuritiesCode', values='Close')

# Impute data
prices_pivot = prices_pivot.interpolate(limit_area='inside')
display(prices_pivot)

# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(prices_pivot)
S = risk_models.CovarianceShrinkage(prices_pivot).ledoit_wolf()

mu.plot.barh(figsize=(10,6))
plotting.plot_covariance(S, plot_correlation=True)

In [ ]:
# Minimal volatility portfolio
ef = EfficientFrontier(mu, S, solver="SCS")
weights_min_vol = ef.min_volatility()
ef.portfolio_performance(verbose=True)
weights_min_vol = pd.DataFrame(weights_min_vol.items()).rename(columns={0:'SecuritiesCode', 1:'Weight'})

In [ ]:
# Tangency(Maximal Sharpe Ratio) Portfolio - Takes long time
#ef = EfficientFrontier(mu, S, solver="SCS")
#weights_max_sharpe = ef.max_sharpe()
#ef.portfolio_performance(verbose=True)
#weights_max_sharpe = pd.DataFrame(weights_max_sharpe.items()).rename(columns={0:'SecuritiesCode', 1:'Weight'})

In [ ]:
def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['Rank'].min() == 0
        assert df['Rank'].max() == len(df['Rank']) - 1
        weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
        short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio

In [ ]:
#Try on data田
weights = weights_min_vol
predict = prices.merge(weights, on='SecuritiesCode', how='left')
predict['Rank'] = predict.groupby('Date')['Weight'].rank(ascending=False)-1
sharpe_ratio = calc_spread_return_sharpe(predict)
display(sharpe_ratio)

In [ ]:
#Submit prediction
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test files
for (prices_test, options_test, financials_test, trades_test, secondary_prices_test, sample_prediction_test) in iter_test:
    predict_test = prices_test.merge(weights, on='SecuritiesCode', how='right')
    predict_test['Rank'] = predict_test.groupby('Date')['Weight'].rank(ascending=False)-1
    sample_prediction_test = predict_test[['Date','SecuritiesCode','Rank']]
    env.predict(sample_prediction_test)   # register your predictions